## 실행 코드

### 0. import library

In [0]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import pickle

#### GPU check

In [0]:
import os
import imageio

if torch.cuda.is_available():
    use_gpu = True
else:
    use_gpu = False
leave_log = True
if leave_log:
    result_dir = '/content/drive/My Drive/Colab deeplearning/GANDAM_bean'
    if not os.path.isdir(result_dir):
        os.mkdir(result_dir)

### 1. 데이터 로드 & 전처리 방식 지정

In [0]:
#데이터 전처리 방식을 지정한다.
transform = transforms.Compose([
  transforms.ToTensor(), # 데이터를 파이토치의 Tensor 형식으로바꾼다.
  transforms.Normalize(mean=(0.5,), std=(0.5,)) # 픽셀값 0 ~ 1 -> -1 ~ 1
])

In [0]:
import glob
import os
import sys

dataPath = "/content/drive/My Drive/bean data"
norm_imgs = glob.glob(os.path.join(dataPath+"/normal",'test_*'))
defect_imgs = glob.glob(os.path.join(dataPath+"/defect",'test_*'))
imgs = norm_imgs + defect_imgs
# imgs

In [0]:
dataPath = "/content/drive/My Drive/bean data"
after_imgs = glob.glob(os.path.join(dataPath+"/after",'test_*'))
imgs = after_imgs
# imgs

In [0]:
dataPath = "/content/drive/My Drive/bean data"
after_imgs = glob.glob(os.path.join(dataPath+"/after_after",'test_*'))
# defect_imgs = glob.glob(os.path.join(dataPath+"/defect",'test_*'))
# imgs = after_imgs + defect_imgs
imgs = after_imgs
# imgs

<hr>

# 코드 정리하기!!!!!

In [0]:
from PIL import Image
import numpy as np
import sklearn
from sklearn.preprocessing import StandardScaler

In [8]:
temp = Image.open(imgs[0])
temp2 = torch.LongTensor(np.array(temp))
temp2.shape

torch.Size([64, 64, 3])

In [0]:
# y_train = np.array([0.]*26 + [1.]*26)
y_train = np.array([0.]*90)
# y_train = np.array([0.]*90 + [1.]*52)
y_train = torch.LongTensor(y_train)

In [55]:
img_arr = list()
# scaler = StandardScaler()
for img in imgs:
  temp = Image.open(img)
  temp = temp.convert("L")
  temp2 = np.array(temp)
  # temp2 =  scaler.fit_transform(np.array(temp))
  img_arr.append(temp2)
  # print(temp2)
  # break
len(img_arr)

90

In [0]:
a = torch.Tensor(np.array(img_arr)).cuda()

In [0]:
q1 = a.unsqueeze(1)

In [58]:
q1.size()

torch.Size([90, 1, 64, 64])

In [0]:
from torch.utils.data import TensorDataset 
from torch.utils.data import DataLoader

In [0]:
dataset = TensorDataset(q1, y_train)

# 코드 정리하기!!!

<hr>


In [0]:
#데이터를 한번에 batch_size만큼만 가져오는 dataloader를 만든다.
# dataloader =DataLoader(dataset, batch_size=10, shuffle=True)
dataloader =DataLoader(dataset, batch_size=30, shuffle=True)

### 2. 모델 구축(생성자 & 구분자)

In [0]:
# 생성자는 랜덤 벡터 z를 입력으로 받아 가짜 이미지를 출력한다.
class Generator(nn.Module):

  # 네트워크 구조
    def __init__(self):
      super(Generator, self).__init__()
      self.main = nn.Sequential(
        nn.Linear(in_features=100, out_features=256),
        nn.LeakyReLU(0.2),
        # nn.Dropout(0.1),
        nn.Linear(in_features=256, out_features=512),
        nn.LeakyReLU(0.2),
        # nn.Dropout(0.1),
        nn.Linear(in_features=512, out_features=1024),
        nn.LeakyReLU(0.2),
        # nn.Dropout(0.1),
        nn.Linear(in_features=1024, out_features=64*64),
        nn.Tanh())
    
  # (batch_size x 100) 크기의 랜덤 벡터를 받아 
  # 이미지를 (batch_size x 1 x 28 x 28) 크기로 출력한다.
    def forward(self, inputs):
      return self.main(inputs).view(-1, 1, 64, 64)

#### 구분자(Discriminator) 구축
  - 이미지를 입력으로 받고 그 이미지가 진짜일 확률을 0과 1 사이의 숫자 하나로 출력하는 함수

#### 사용하는 모델
- 4개의 선형 레이어
  - 이미지(28x28) => 레이어(1024 뉴런)  
    => 레이어(512) => 레이어(256) => 마지막 예측 레이어(1)
- 활성 함수로는 LeakyReLU ( 0보다 낮은 값은 0.2(설정값)을 곱 )
  - 마지막 레이어 : Sigmoid ( 출략값이 0~1 이기 때문에 )
- 드롭아웃 층
  - 과적합(Overfitting, 오버피팅)되는 것을 방지
  - 구분자가 생성자보다 지나치게 빨리 학습되는 것 막기 위함.

In [0]:
# 구분자는 이미지를 입력으로 받아 이미지가 진짜인지 가짜인지 출력한다.
class Discriminator(nn.Module):
    
# 네트워크 구조
  def __init__(self):
    super(Discriminator, self).__init__()
    self.main = nn.Sequential(
      nn.Linear(in_features=64*64, out_features=1024),
      nn.LeakyReLU(0.1, ),
      # nn.Dropout(0.1),
      nn.Linear(in_features=1024, out_features=512),
      nn.LeakyReLU(0.1, ),
      # nn.Dropout(0.1),
      nn.Linear(in_features=512, out_features=256),
      nn.LeakyReLU(0.1, ),
      # nn.Dropout(0.1),
      nn.Linear(in_features=256, out_features=1),
      nn.Sigmoid())
    
  # (batch_size x 1 x 28 x 28) 크기의 이미지를 받아
  # 이미지가 진짜일 확률을 0~1 사이로 출력한다.
  def forward(self, inputs):
    inputs = inputs.view(-1, 64*64)
    return self.main(inputs)

#### 생성자 & 구분자 객체 생성

In [64]:
G = Generator()
D = Discriminator()

if use_gpu:
    print("using_GPU")
    G.cuda()
    D.cuda()

using_GPU


### 손실함수 & 최적화기법 지정

- 손실 함수
  - Binary Cross Entropy Loss function
    - 확률이 정답에 가까워지면 낮아지고, 멀면 높아진다.
    - 낮추는 것이 목표
- 최적화 기법 
  - Adam
    - 매개변수마다 업데이트 속도를 최적으로 조절하는 효육적인 최적화 기법

In [0]:
# Binary Cross Entropy loss
# criterion = nn.BCELoss()
# criterion = nn.BCEWithLogitsLoss()
criterion = nn.BCELoss()

# 생성자의 매개 변수를 최적화하는 Adam optimizer
G_optimizer = Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))
# 구분자의 매개 변수를 최적화하는 Adam optimizer
D_optimizer = Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))

### 시각화 함수

In [0]:
# 학습 결과 시각화하기
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np

def square_plot(data, path):
    """Take an array of shape (n, height, width) or (n, height, width , 3)
       and visualize each (height, width) thing in a grid of size approx. sqrt(n) by sqrt(n)"""

    if type(data) == list:
	    data = np.concatenate(data)
    # normalize data for display
    data = (data - data.min()) / (data.max() - data.min())

    # force the number of filters to be square
    n = int(np.ceil(np.sqrt(data.shape[0])))

    padding = (((0, n ** 2 - data.shape[0]) ,
                (0, 1), (0, 1))  # add some space between filters
               + ((0, 0),) * (data.ndim - 3))  # don't pad the last dimension (if there is one)
    data = np.pad(data , padding, mode='constant' , constant_values=1)  # pad with ones (white)

    # tilethe filters into an image
    data = data.reshape((n , n) + data.shape[1:]).transpose((0 , 2 , 1 , 3) + tuple(range(4 , data.ndim + 1)))

    data = data.reshape((n * data.shape[1] , n * data.shape[3]) + data.shape[4:])

    plt.imsave(path, data, cmap='gray')

In [52]:
if leave_log:
    train_hist = {}
    train_hist['D_losses'] = []
    train_hist['G_losses'] = []
    generated_images = []
    
z_fixed = Variable(torch.randn(5 * 5, 100), volatile=True)
if use_gpu:
    z_fixed = z_fixed.cuda()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  import sys


### 모델 반복학습

- 100 epoch  
  => 데이터셋을 100번 순회
- 60 batch size  
  => epoch마다 60개의 데이터를 가져오겠다.

MNIST 학습 데이터의 개수가 6만개이니 1에폭마다 1000번씩 학습이 이루어지는 셈이다.

In [65]:
from time import time

start = time()
# 데이터셋을 100번 돌며 학습한다.
for epoch in range(100):
    
    if leave_log:
        D_losses = []
        G_losses = []
    
    # 한번에 batch_size만큼 데이터를 가져온다.
    for real_data, _ in dataloader:
        batch_size = real_data.size(0)
        # print(batch_size)
        
        # 데이터를 pytorch의 변수로 변환한다.
        real_data = Variable(real_data)
        # print(real_data.size())

        ### 구분자 학습시키기

        # 이미지가 진짜일 때 정답 값은 1이고 가짜일 때는 0이다.
        # 정답지에 해당하는 변수를 만든다.
        target_real = Variable(torch.ones(batch_size, 1))
        target_fake = Variable(torch.zeros(batch_size, 1))
         
        if use_gpu:
            real_data, target_real, target_fake = real_data.cuda(), target_real.cuda(), target_fake.cuda()
            
        # 진짜 이미지를 구분자에 넣는다.
        D_result_from_real = D(real_data)
        # 구분자의 출력값이 정답지인 1에서 멀수록 loss가 높아진다.
        D_loss_real = criterion(D_result_from_real, target_real)

        # 생성자에 입력으로 줄 랜덤 벡터 z를 만든다.
        z = Variable(torch.randn((batch_size, 100)))
        
        if use_gpu:
            z = z.cuda()
            
        # 생성자로 가짜 이미지를 생성한다.
        fake_data = G(z)
        
        # 생성자가 만든 가짜 이미지를 구분자에 넣는다.
        D_result_from_fake = D(fake_data)
        # 구분자의 출력값이 정답지인 0에서 멀수록 loss가 높아진다.
        D_loss_fake = criterion(D_result_from_fake, target_fake)
        
        # 구분자의 loss는 두 문제에서 계산된 loss의 합이다.
        D_loss = D_loss_real + D_loss_fake
        
        # 구분자의 매개 변수의 미분값을 0으로 초기화한다.
        D.zero_grad()
        # 역전파를 통해 매개 변수의 loss에 대한 미분값을 계산한다.
        D_loss.backward()
        # 최적화 기법을 이용해 구분자의 매개 변수를 업데이트한다.
        D_optimizer.step()
        
        if leave_log:
            # D_losses.append(D_loss.data[0])
            D_losses.append(D_loss.item())

        # train generator G

        ### 생성자 학습시키기
        
        # 생성자에 입력으로 줄 랜덤 벡터 z를 만든다.
        z = Variable(torch.randn((batch_size, 100)))
        
        if use_gpu:
            z = z.cuda()
        
        # 생성자로 가짜 이미지를 생성한다.
        fake_data = G(z)
        # 생성자가 만든 가짜 이미지를 구분자에 넣는다.
        D_result_from_fake = D(fake_data)
        # 생성자의 입장에서 구분자의 출력값이 1에서 멀수록 loss가 높아진다.
        G_loss = criterion(D_result_from_fake, target_real)
        
        # 생성자의 매개 변수의 미분값을 0으로 초기화한다.
        G.zero_grad()
        # 역전파를 통해 매개 변수의 loss에 대한 미분값을 계산한다.
        G_loss.backward()
        # 최적화 기법을 이용해 생성자의 매개 변수를 업데이트한다.
        G_optimizer.step()
        
        if leave_log:
            # G_losses.append(G_loss.data[0])
            G_losses.append(G_loss.item())
    if leave_log:
        # true_positive_rate = (D_result_from_real > 0.5).float().mean().data[0]
        # true_negative_rate = (D_result_from_fake < 0.5).float().mean().data[0]
        true_positive_rate = (D_result_from_real > 0.5).float().mean().item()
        true_negative_rate = (D_result_from_fake < 0.5).float().mean().item()
        base_message = ("Epoch: {epoch:<3d} D Loss: {d_loss:<8.6} G Loss: {g_loss:<8.6} "
                        "True Positive Rate: {tpr:<5.1%} True Negative Rate: {tnr:<5.1%}"
                       )
        message = base_message.format(
                    epoch=epoch,
                    d_loss=sum(D_losses)/len(D_losses),
                    g_loss=sum(G_losses)/len(G_losses),
                    tpr=true_positive_rate,
                    tnr=true_negative_rate
        )
        print(message)
    
    if leave_log:
        fake_data_fixed = G(z_fixed)
        image_path = result_dir + '/epoch{}.png'.format(epoch)
        square_plot(fake_data_fixed.view(25, 64, 64).cpu().data.numpy(), path=image_path)
        generated_images.append(image_path)
    
    if leave_log:
        train_hist['D_losses'].append(torch.mean(torch.FloatTensor(D_losses)))
        train_hist['G_losses'].append(torch.mean(torch.FloatTensor(G_losses)))

torch.save(G.state_dict(), "gan_generator.pkl")
torch.save(D.state_dict(), "gan_discriminator.pkl")
with open('gan_train_history.pkl', 'wb') as f:
    pickle.dump(train_hist, f)

generated_image_array = [imageio.imread(generated_image) for generated_image in generated_images]
imageio.mimsave(result_dir + '/GAN_generation.gif', generated_image_array, fps=5)

print("end_time : ", str(start-time()))

Epoch: 0   D Loss: 3.35653  G Loss: 0.707976 True Positive Rate: 3.3%  True Negative Rate: 100.0%
Epoch: 1   D Loss: 3.3565   G Loss: 0.707965 True Positive Rate: 0.0%  True Negative Rate: 100.0%
Epoch: 2   D Loss: 3.35655  G Loss: 0.707918 True Positive Rate: 0.0%  True Negative Rate: 100.0%
Epoch: 3   D Loss: 3.35651  G Loss: 0.707994 True Positive Rate: 0.0%  True Negative Rate: 100.0%
Epoch: 4   D Loss: 3.35664  G Loss: 0.707891 True Positive Rate: 0.0%  True Negative Rate: 100.0%
Epoch: 5   D Loss: 3.35669  G Loss: 0.707942 True Positive Rate: 3.3%  True Negative Rate: 100.0%
Epoch: 6   D Loss: 3.3565   G Loss: 0.707926 True Positive Rate: 3.3%  True Negative Rate: 100.0%
Epoch: 7   D Loss: 3.35657  G Loss: 0.707957 True Positive Rate: 0.0%  True Negative Rate: 100.0%
Epoch: 8   D Loss: 3.35655  G Loss: 0.708028 True Positive Rate: 0.0%  True Negative Rate: 100.0%
Epoch: 9   D Loss: 3.3565   G Loss: 0.707993 True Positive Rate: 0.0%  True Negative Rate: 100.0%
Epoch: 10  D Loss: 3

/usr/local/lib/python3.6/dist-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


KeyboardInterrupt: ignored

In [0]:
z = Variable(torch.randn((batch_size, 80)))
z

tensor([[ 1.1534,  1.4060,  1.0578,  ..., -2.3254,  0.3431,  1.3877],
        [ 1.3626,  1.8769,  1.2459,  ...,  0.0521,  1.2031, -1.5178],
        [ 0.0888,  0.9181, -0.9085,  ..., -1.6978,  1.5547, -2.3547],
        ...,
        [ 0.3066,  0.8023,  0.2619,  ...,  1.0835, -0.1802, -0.5282],
        [-0.7603,  0.1801,  1.6961,  ..., -0.0572,  1.0028,  0.8055],
        [ 0.8609,  0.0818,  0.4082,  ..., -1.5827, -1.1080, -0.7029]])

In [0]:
import random
random.random()

0.7635124661404858

# rand int(0~1)

- scaler => 정규분포 교체
- vector => randn 교체

~~~
Epoch: 0   D Loss: 0.990409 G Loss: 1.38524  True Positive Rate: 46.7% True Negative Rate: 100.0%
Epoch: 1   D Loss: 1.27203  G Loss: 1.60394  True Positive Rate: 50.0% True Negative Rate: 100.0%
Epoch: 2   D Loss: 1.58664  G Loss: 1.18453  True Positive Rate: 30.0% True Negative Rate: 93.3%
Epoch: 3   D Loss: 1.12798  G Loss: 1.36241  True Positive Rate: 63.3% True Negative Rate: 100.0%
Epoch: 4   D Loss: 1.65854  G Loss: 1.83637  True Positive Rate: 53.3% True Negative Rate: 100.0%
Epoch: 5   D Loss: 1.25826  G Loss: 1.66822  True Positive Rate: 100.0% True Negative Rate: 100.0%
Epoch: 6   D Loss: 1.60527  G Loss: 1.40425  True Positive Rate: 53.3% True Negative Rate: 73.3%
Epoch: 7   D Loss: 1.87918  G Loss: 0.761097 True Positive Rate: 56.7% True Negative Rate: 76.7%
Epoch: 8   D Loss: 1.22751  G Loss: 1.24992  True Positive Rate: 43.3% True Negative Rate: 100.0%
Epoch: 9   D Loss: 1.20484  G Loss: 1.26276  True Positive Rate: 56.7% True Negative Rate: 100.0%
Epoch: 10  D Loss: 1.35906  G Loss: 1.27616  True Positive Rate: 23.3% True Negative Rate: 100.0%
Epoch: 11  D Loss: 1.43436  G Loss: 1.66363  True Positive Rate: 50.0% True Negative Rate: 100.0%
Epoch: 12  D Loss: 2.06688  G Loss: 1.77353  True Positive Rate: 0.0%  True Negative Rate: 100.0%
Epoch: 13  D Loss: 1.82243  G Loss: 2.01021  True Positive Rate: 56.7% True Negative Rate: 100.0%
Epoch: 14  D Loss: 0.942322 G Loss: 2.00392  True Positive Rate: 63.3% True Negative Rate: 100.0%
Epoch: 15  D Loss: 1.00432  G Loss: 1.65147  True Positive Rate: 60.0% True Negative Rate: 100.0%
Epoch: 16  D Loss: 1.46306  G Loss: 1.13022  True Positive Rate: 40.0% True Negative Rate: 100.0%
Epoch: 17  D Loss: 1.69429  G Loss: 0.822099 True Positive Rate: 43.3% True Negative Rate: 100.0%
Epoch: 18  D Loss: 1.59064  G Loss: 0.787313 True Positive Rate: 40.0% True Negative Rate: 100.0%
Epoch: 19  D Loss: 1.48329  G Loss: 0.919296 True Positive Rate: 40.0% True Negative Rate: 100.0%
Epoch: 20  D Loss: 1.31983  G Loss: 1.08266  True Positive Rate: 53.3% True Negative Rate: 100.0%
Epoch: 21  D Loss: 1.14992  G Loss: 1.28387  True Positive Rate: 66.7% True Negative Rate: 100.0%
Epoch: 22  D Loss: 1.0942   G Loss: 1.4536   True Positive Rate: 56.7% True Negative Rate: 100.0%
Epoch: 23  D Loss: 1.44834  G Loss: 1.72904  True Positive Rate: 46.7% True Negative Rate: 100.0%
Epoch: 24  D Loss: 1.73717  G Loss: 1.51053  True Positive Rate: 23.3% True Negative Rate: 100.0%
Epoch: 25  D Loss: 1.80419  G Loss: 1.43129  True Positive Rate: 33.3% True Negative Rate: 100.0%
Epoch: 26  D Loss: 1.92125  G Loss: 1.37885  True Positive Rate: 6.7%  True Negative Rate: 100.0%
Epoch: 27  D Loss: 1.53847  G Loss: 1.38698  True Positive Rate: 40.0% True Negative Rate: 100.0%
Epoch: 28  D Loss: 0.995595 G Loss: 1.70165  True Positive Rate: 73.3% True Negative Rate: 100.0%
Epoch: 29  D Loss: 1.06438  G Loss: 1.55487  True Positive Rate: 73.3% True Negative Rate: 100.0%
Epoch: 30  D Loss: 1.36968  G Loss: 1.15213  True Positive Rate: 73.3% True Negative Rate: 100.0%
Epoch: 31  D Loss: 1.60984  G Loss: 0.955796 True Positive Rate: 56.7% True Negative Rate: 100.0%
Epoch: 32  D Loss: 1.69537  G Loss: 0.743417 True Positive Rate: 46.7% True Negative Rate: 90.0%
Epoch: 33  D Loss: 1.53447  G Loss: 0.765797 True Positive Rate: 66.7% True Negative Rate: 100.0%
Epoch: 34  D Loss: 1.36583  G Loss: 0.947653 True Positive Rate: 46.7% True Negative Rate: 100.0%
Epoch: 35  D Loss: 1.31552  G Loss: 1.07511  True Positive Rate: 46.7% True Negative Rate: 100.0%
Epoch: 36  D Loss: 1.38344  G Loss: 1.10122  True Positive Rate: 46.7% True Negative Rate: 100.0%
Epoch: 37  D Loss: 1.53153  G Loss: 1.12254  True Positive Rate: 30.0% True Negative Rate: 100.0%
Epoch: 38  D Loss: 1.53107  G Loss: 1.15483  True Positive Rate: 40.0% True Negative Rate: 100.0%
Epoch: 39  D Loss: 1.61191  G Loss: 1.38747  True Positive Rate: 20.0% True Negative Rate: 100.0%
Epoch: 40  D Loss: 1.61929  G Loss: 1.50426  True Positive Rate: 16.7% True Negative Rate: 100.0%
Epoch: 41  D Loss: 1.31622  G Loss: 1.6568   True Positive Rate: 20.0% True Negative Rate: 100.0%
Epoch: 42  D Loss: 1.09822  G Loss: 1.78888  True Positive Rate: 70.0% True Negative Rate: 100.0%
Epoch: 43  D Loss: 1.25877  G Loss: 1.57229  True Positive Rate: 60.0% True Negative Rate: 100.0%
Epoch: 44  D Loss: 1.31139  G Loss: 1.42331  True Positive Rate: 60.0% True Negative Rate: 100.0%
Epoch: 45  D Loss: 1.46987  G Loss: 1.16977  True Positive Rate: 46.7% True Negative Rate: 100.0%
Epoch: 46  D Loss: 1.47163  G Loss: 0.937221 True Positive Rate: 33.3% True Negative Rate: 100.0%
Epoch: 47  D Loss: 1.32611  G Loss: 0.918541 True Positive Rate: 36.7% True Negative Rate: 100.0%
Epoch: 48  D Loss: 1.21565  G Loss: 0.947456 True Positive Rate: 60.0% True Negative Rate: 100.0%
Epoch: 49  D Loss: 1.16054  G Loss: 1.03136  True Positive Rate: 80.0% True Negative Rate: 100.0%
Epoch: 50  D Loss: 1.18133  G Loss: 0.961517 True Positive Rate: 56.7% True Negative Rate: 100.0%
Epoch: 51  D Loss: 1.20286  G Loss: 0.968519 True Positive Rate: 56.7% True Negative Rate: 100.0%
Epoch: 52  D Loss: 1.19391  G Loss: 0.899091 True Positive Rate: 56.7% True Negative Rate: 100.0%
Epoch: 53  D Loss: 1.17388  G Loss: 0.931549 True Positive Rate: 73.3% True Negative Rate: 100.0%
Epoch: 54  D Loss: 1.13433  G Loss: 0.958241 True Positive Rate: 56.7% True Negative Rate: 100.0%
Epoch: 55  D Loss: 1.05674  G Loss: 1.05889  True Positive Rate: 70.0% True Negative Rate: 100.0%
Epoch: 56  D Loss: 0.935457 G Loss: 1.1455   True Positive Rate: 73.3% True Negative Rate: 100.0%
Epoch: 57  D Loss: 0.980629 G Loss: 1.28014  True Positive Rate: 96.7% True Negative Rate: 100.0%
Epoch: 58  D Loss: 1.04478  G Loss: 1.37386  True Positive Rate: 76.7% True Negative Rate: 100.0%
Epoch: 59  D Loss: 1.08758  G Loss: 1.35638  True Positive Rate: 66.7% True Negative Rate: 100.0%
Epoch: 60  D Loss: 0.906913 G Loss: 1.43278  True Positive Rate: 76.7% True Negative Rate: 100.0%
Epoch: 61  D Loss: 0.833566 G Loss: 1.49659  True Positive Rate: 83.3% True Negative Rate: 100.0%
Epoch: 62  D Loss: 0.974353 G Loss: 1.24301  True Positive Rate: 86.7% True Negative Rate: 100.0%
Epoch: 63  D Loss: 1.03351  G Loss: 1.07428  True Positive Rate: 63.3% True Negative Rate: 100.0%
Epoch: 64  D Loss: 1.07226  G Loss: 1.0666   True Positive Rate: 53.3% True Negative Rate: 100.0%
Epoch: 65  D Loss: 1.10038  G Loss: 1.20801  True Positive Rate: 63.3% True Negative Rate: 100.0%
Epoch: 66  D Loss: 1.05886  G Loss: 1.13145  True Positive Rate: 63.3% True Negative Rate: 100.0%
Epoch: 67  D Loss: 0.953486 G Loss: 1.16276  True Positive Rate: 70.0% True Negative Rate: 100.0%
Epoch: 68  D Loss: 0.945249 G Loss: 1.3091   True Positive Rate: 46.7% True Negative Rate: 100.0%
Epoch: 69  D Loss: 1.07632  G Loss: 1.45596  True Positive Rate: 56.7% True Negative Rate: 100.0%
Epoch: 70  D Loss: 1.22053  G Loss: 1.49016  True Positive Rate: 50.0% True Negative Rate: 100.0%
Epoch: 71  D Loss: 1.42419  G Loss: 1.58302  True Positive Rate: 13.3% True Negative Rate: 100.0%
Epoch: 72  D Loss: 1.30198  G Loss: 1.30961  True Positive Rate: 46.7% True Negative Rate: 100.0%
Epoch: 73  D Loss: 1.03926  G Loss: 1.48842  True Positive Rate: 83.3% True Negative Rate: 100.0%
Epoch: 74  D Loss: 0.910021 G Loss: 1.40388  True Positive Rate: 76.7% True Negative Rate: 100.0%
Epoch: 75  D Loss: 0.879891 G Loss: 1.30806  True Positive Rate: 73.3% True Negative Rate: 100.0%
Epoch: 76  D Loss: 1.02487  G Loss: 1.14405  True Positive Rate: 63.3% True Negative Rate: 100.0%
Epoch: 77  D Loss: 1.19096  G Loss: 0.968821 True Positive Rate: 70.0% True Negative Rate: 100.0%
Epoch: 78  D Loss: 1.24043  G Loss: 0.851859 True Positive Rate: 73.3% True Negative Rate: 100.0%
Epoch: 79  D Loss: 1.36529  G Loss: 0.86162  True Positive Rate: 66.7% True Negative Rate: 100.0%
Epoch: 80  D Loss: 1.47393  G Loss: 0.77695  True Positive Rate: 46.7% True Negative Rate: 100.0%
Epoch: 81  D Loss: 1.48082  G Loss: 0.768555 True Positive Rate: 53.3% True Negative Rate: 100.0%
Epoch: 82  D Loss: 1.39429  G Loss: 0.964111 True Positive Rate: 53.3% True Negative Rate: 100.0%
Epoch: 83  D Loss: 1.26467  G Loss: 1.16427  True Positive Rate: 63.3% True Negative Rate: 100.0%
Epoch: 84  D Loss: 1.27106  G Loss: 1.33496  True Positive Rate: 66.7% True Negative Rate: 100.0%
Epoch: 85  D Loss: 1.57101  G Loss: 1.55991  True Positive Rate: 40.0% True Negative Rate: 100.0%
Epoch: 86  D Loss: 1.76473  G Loss: 1.57733  True Positive Rate: 23.3% True Negative Rate: 100.0%
Epoch: 87  D Loss: 1.65184  G Loss: 1.76637  True Positive Rate: 20.0% True Negative Rate: 100.0%
Epoch: 88  D Loss: 1.44276  G Loss: 1.88189  True Positive Rate: 33.3% True Negative Rate: 100.0%
Epoch: 89  D Loss: 1.35472  G Loss: 2.11164  True Positive Rate: 40.0% True Negative Rate: 100.0%
Epoch: 90  D Loss: 1.47351  G Loss: 1.9533   True Positive Rate: 36.7% True Negative Rate: 100.0%
Epoch: 91  D Loss: 1.67061  G Loss: 1.41472  True Positive Rate: 10.0% True Negative Rate: 100.0%
Epoch: 92  D Loss: 1.47795  G Loss: 1.16351  True Positive Rate: 30.0% True Negative Rate: 100.0%
Epoch: 93  D Loss: 1.0964   G Loss: 1.36057  True Positive Rate: 46.7% True Negative Rate: 100.0%
Epoch: 94  D Loss: 0.944807 G Loss: 1.52948  True Positive Rate: 56.7% True Negative Rate: 100.0%
Epoch: 95  D Loss: 1.09354  G Loss: 1.10983  True Positive Rate: 70.0% True Negative Rate: 100.0%
Epoch: 96  D Loss: 1.26797  G Loss: 0.978764 True Positive Rate: 56.7% True Negative Rate: 100.0%
Epoch: 97  D Loss: 1.22352  G Loss: 1.04862  True Positive Rate: 56.7% True Negative Rate: 100.0%
Epoch: 98  D Loss: 1.11339  G Loss: 1.1801   True Positive Rate: 53.3% True Negative Rate: 100.0%
Epoch: 99  D Loss: 1.10983  G Loss: 1.20909  True Positive Rate: 66.7% True Negative Rate: 100.0%
Epoch: 100 D Loss: 1.05326  G Loss: 1.30998  True Positive Rate: 66.7% True Negative Rate: 100.0%
Epoch: 101 D Loss: 1.19563  G Loss: 1.4147   True Positive Rate: 60.0% True Negative Rate: 100.0%
Epoch: 102 D Loss: 1.24383  G Loss: 1.25892  True Positive Rate: 70.0% True Negative Rate: 100.0%
Epoch: 103 D Loss: 1.01744  G Loss: 1.35167  True Positive Rate: 73.3% True Negative Rate: 100.0%
Epoch: 104 D Loss: 1.10814  G Loss: 1.46978  True Positive Rate: 76.7% True Negative Rate: 100.0%
Epoch: 105 D Loss: 1.48636  G Loss: 1.12243  True Positive Rate: 50.0% True Negative Rate: 100.0%
Epoch: 106 D Loss: 1.35668  G Loss: 0.87474  True Positive Rate: 76.7% True Negative Rate: 100.0%
Epoch: 107 D Loss: 1.12039  G Loss: 1.01572  True Positive Rate: 73.3% True Negative Rate: 100.0%
Epoch: 108 D Loss: 0.999875 G Loss: 1.1419   True Positive Rate: 70.0% True Negative Rate: 100.0%
Epoch: 109 D Loss: 1.01567  G Loss: 1.22925  True Positive Rate: 43.3% True Negative Rate: 100.0%
Epoch: 110 D Loss: 1.09361  G Loss: 1.17642  True Positive Rate: 76.7% True Negative Rate: 100.0%
Epoch: 111 D Loss: 1.20503  G Loss: 1.10615  True Positive Rate: 53.3% True Negative Rate: 100.0%
Epoch: 112 D Loss: 1.26107  G Loss: 1.08401  True Positive Rate: 60.0% True Negative Rate: 100.0%
Epoch: 113 D Loss: 1.24478  G Loss: 1.04437  True Positive Rate: 53.3% True Negative Rate: 100.0%
Epoch: 114 D Loss: 1.12806  G Loss: 1.15327  True Positive Rate: 60.0% True Negative Rate: 100.0%
Epoch: 115 D Loss: 0.99158  G Loss: 1.33976  True Positive Rate: 80.0% True Negative Rate: 100.0%
Epoch: 116 D Loss: 0.936419 G Loss: 1.33594  True Positive Rate: 83.3% True Negative Rate: 100.0%
Epoch: 117 D Loss: 0.991113 G Loss: 1.58834  True Positive Rate: 100.0% True Negative Rate: 100.0%
Epoch: 118 D Loss: 1.00892  G Loss: 1.38297  True Positive Rate: 96.7% True Negative Rate: 100.0%
Epoch: 119 D Loss: 1.04912  G Loss: 1.27545  True Positive Rate: 86.7% True Negative Rate: 100.0%
Epoch: 120 D Loss: 1.04064  G Loss: 1.26572  True Positive Rate: 90.0% True Negative Rate: 100.0%
Epoch: 121 D Loss: 0.830799 G Loss: 1.21753  True Positive Rate: 100.0% True Negative Rate: 100.0%
Epoch: 122 D Loss: 0.631105 G Loss: 1.35053  True Positive Rate: 93.3% True Negative Rate: 100.0%
Epoch: 123 D Loss: 0.713063 G Loss: 1.38922  True Positive Rate: 86.7% True Negative Rate: 100.0%
Epoch: 124 D Loss: 0.728369 G Loss: 1.3948   True Positive Rate: 80.0% True Negative Rate: 100.0%
Epoch: 125 D Loss: 0.89531  G Loss: 1.33786  True Positive Rate: 70.0% True Negative Rate: 100.0%
Epoch: 126 D Loss: 1.49208  G Loss: 1.09694  True Positive Rate: 36.7% True Negative Rate: 100.0%
Epoch: 127 D Loss: 1.55071  G Loss: 1.13239  True Positive Rate: 60.0% True Negative Rate: 100.0%
Epoch: 128 D Loss: 1.51611  G Loss: 1.03071  True Positive Rate: 46.7% True Negative Rate: 100.0%
Epoch: 129 D Loss: 1.43756  G Loss: 0.95797  True Positive Rate: 46.7% True Negative Rate: 100.0%
Epoch: 130 D Loss: 1.22826  G Loss: 1.06183  True Positive Rate: 50.0% True Negative Rate: 100.0%
Epoch: 131 D Loss: 0.979778 G Loss: 1.25856  True Positive Rate: 80.0% True Negative Rate: 100.0%
Epoch: 132 D Loss: 0.83276  G Loss: 1.50409  True Positive Rate: 83.3% True Negative Rate: 100.0%
Epoch: 133 D Loss: 0.918514 G Loss: 1.62083  True Positive Rate: 86.7% True Negative Rate: 100.0%
Epoch: 134 D Loss: 1.38293  G Loss: 1.49804  True Positive Rate: 13.3% True Negative Rate: 100.0%
Epoch: 135 D Loss: 1.42486  G Loss: 1.13696  True Positive Rate: 30.0% True Negative Rate: 100.0%
Epoch: 136 D Loss: 1.03438  G Loss: 1.07715  True Positive Rate: 100.0% True Negative Rate: 100.0%
Epoch: 137 D Loss: 0.945063 G Loss: 1.26919  True Positive Rate: 80.0% True Negative Rate: 100.0%
Epoch: 138 D Loss: 1.1709   G Loss: 1.21613  True Positive Rate: 50.0% True Negative Rate: 100.0%
Epoch: 139 D Loss: 1.21785  G Loss: 1.04643  True Positive Rate: 63.3% True Negative Rate: 100.0%
Epoch: 140 D Loss: 1.17296  G Loss: 1.18513  True Positive Rate: 50.0% True Negative Rate: 100.0%
Epoch: 141 D Loss: 1.16436  G Loss: 1.24095  True Positive Rate: 50.0% True Negative Rate: 100.0%
Epoch: 142 D Loss: 1.18972  G Loss: 1.32905  True Positive Rate: 30.0% True Negative Rate: 100.0%
Epoch: 143 D Loss: 1.18876  G Loss: 1.19391  True Positive Rate: 46.7% True Negative Rate: 100.0%
Epoch: 144 D Loss: 1.14434  G Loss: 1.11968  True Positive Rate: 66.7% True Negative Rate: 100.0%
Epoch: 145 D Loss: 1.04499  G Loss: 1.13418  True Positive Rate: 76.7% True Negative Rate: 100.0%
Epoch: 146 D Loss: 0.925552 G Loss: 1.31303  True Positive Rate: 80.0% True Negative Rate: 100.0%
Epoch: 147 D Loss: 0.86291  G Loss: 1.36935  True Positive Rate: 76.7% True Negative Rate: 100.0%
Epoch: 148 D Loss: 0.813161 G Loss: 1.37835  True Positive Rate: 73.3% True Negative Rate: 100.0%
Epoch: 149 D Loss: 0.864099 G Loss: 1.43637  True Positive Rate: 50.0% True Negative Rate: 100.0%
Epoch: 150 D Loss: 1.43143  G Loss: 1.41343  True Positive Rate: 30.0% True Negative Rate: 100.0%
Epoch: 151 D Loss: 1.92235  G Loss: 0.892886 True Positive Rate: 23.3% True Negative Rate: 83.3%
Epoch: 152 D Loss: 1.98533  G Loss: 0.626468 True Positive Rate: 20.0% True Negative Rate: 3.3% 
Epoch: 153 D Loss: 1.75206  G Loss: 0.735035 True Positive Rate: 33.3% True Negative Rate: 100.0%
Epoch: 154 D Loss: 1.48477  G Loss: 0.932511 True Positive Rate: 46.7% True Negative Rate: 100.0%
Epoch: 155 D Loss: 1.3253   G Loss: 1.0705   True Positive Rate: 60.0% True Negative Rate: 100.0%
Epoch: 156 D Loss: 1.2677   G Loss: 1.15755  True Positive Rate: 70.0% True Negative Rate: 100.0%
Epoch: 157 D Loss: 1.32031  G Loss: 1.16855  True Positive Rate: 76.7% True Negative Rate: 100.0%
Epoch: 158 D Loss: 1.39323  G Loss: 1.18267  True Positive Rate: 36.7% True Negative Rate: 100.0%
Epoch: 159 D Loss: 1.46966  G Loss: 1.09076  True Positive Rate: 56.7% True Negative Rate: 100.0%
Epoch: 160 D Loss: 1.43362  G Loss: 1.09079  True Positive Rate: 46.7% True Negative Rate: 100.0%
Epoch: 161 D Loss: 1.38274  G Loss: 1.06737  True Positive Rate: 40.0% True Negative Rate: 100.0%
Epoch: 162 D Loss: 1.32184  G Loss: 1.10149  True Positive Rate: 53.3% True Negative Rate: 100.0%
Epoch: 163 D Loss: 1.15025  G Loss: 1.19762  True Positive Rate: 63.3% True Negative Rate: 100.0%
Epoch: 164 D Loss: 0.992689 G Loss: 1.27865  True Positive Rate: 66.7% True Negative Rate: 100.0%
Epoch: 165 D Loss: 0.978424 G Loss: 1.2633   True Positive Rate: 83.3% True Negative Rate: 100.0%
Epoch: 166 D Loss: 1.03536  G Loss: 1.12888  True Positive Rate: 80.0% True Negative Rate: 100.0%
Epoch: 167 D Loss: 1.40182  G Loss: 1.01998  True Positive Rate: 50.0% True Negative Rate: 100.0%
Epoch: 168 D Loss: 1.72167  G Loss: 0.831796 True Positive Rate: 26.7% True Negative Rate: 100.0%
Epoch: 169 D Loss: 1.44017  G Loss: 0.734123 True Positive Rate: 80.0% True Negative Rate: 66.7%
Epoch: 170 D Loss: 1.04735  G Loss: 1.03696  True Positive Rate: 80.0% True Negative Rate: 100.0%
Epoch: 171 D Loss: 0.854695 G Loss: 1.30731  True Positive Rate: 76.7% True Negative Rate: 100.0%
Epoch: 172 D Loss: 0.921275 G Loss: 1.43761  True Positive Rate: 86.7% True Negative Rate: 100.0%
Epoch: 173 D Loss: 1.09996  G Loss: 1.21177  True Positive Rate: 63.3% True Negative Rate: 100.0%
Epoch: 174 D Loss: 1.11126  G Loss: 1.11736  True Positive Rate: 46.7% True Negative Rate: 100.0%
Epoch: 175 D Loss: 1.11037  G Loss: 1.04184  True Positive Rate: 53.3% True Negative Rate: 100.0%
Epoch: 176 D Loss: 1.06262  G Loss: 1.07149  True Positive Rate: 60.0% True Negative Rate: 100.0%
Epoch: 177 D Loss: 1.09627  G Loss: 1.06586  True Positive Rate: 60.0% True Negative Rate: 100.0%
Epoch: 178 D Loss: 1.04001  G Loss: 1.19953  True Positive Rate: 70.0% True Negative Rate: 100.0%
Epoch: 179 D Loss: 1.0151   G Loss: 1.14341  True Positive Rate: 66.7% True Negative Rate: 100.0%
Epoch: 180 D Loss: 0.922163 G Loss: 1.25886  True Positive Rate: 76.7% True Negative Rate: 100.0%
Epoch: 181 D Loss: 0.884992 G Loss: 1.26936  True Positive Rate: 60.0% True Negative Rate: 100.0%
Epoch: 182 D Loss: 0.768008 G Loss: 1.30016  True Positive Rate: 76.7% True Negative Rate: 100.0%
Epoch: 183 D Loss: 0.682484 G Loss: 1.28988  True Positive Rate: 93.3% True Negative Rate: 100.0%
Epoch: 184 D Loss: 0.635521 G Loss: 1.34204  True Positive Rate: 83.3% True Negative Rate: 100.0%
Epoch: 185 D Loss: 0.647937 G Loss: 1.3585   True Positive Rate: 86.7% True Negative Rate: 100.0%
Epoch: 186 D Loss: 0.665769 G Loss: 1.31946  True Positive Rate: 93.3% True Negative Rate: 100.0%
Epoch: 187 D Loss: 0.688987 G Loss: 1.30906  True Positive Rate: 83.3% True Negative Rate: 100.0%
Epoch: 188 D Loss: 0.803481 G Loss: 1.14138  True Positive Rate: 90.0% True Negative Rate: 100.0%
Epoch: 189 D Loss: 0.950419 G Loss: 1.00811  True Positive Rate: 73.3% True Negative Rate: 100.0%
Epoch: 190 D Loss: 1.39841  G Loss: 0.716276 True Positive Rate: 60.0% True Negative Rate: 36.7%
Epoch: 191 D Loss: 1.31047  G Loss: 0.829697 True Positive Rate: 96.7% True Negative Rate: 100.0%
Epoch: 192 D Loss: 0.831449 G Loss: 1.60141  True Positive Rate: 100.0% True Negative Rate: 100.0%
Epoch: 193 D Loss: 0.973539 G Loss: 1.96865  True Positive Rate: 60.0% True Negative Rate: 100.0%
Epoch: 194 D Loss: 1.22514  G Loss: 1.49774  True Positive Rate: 53.3% True Negative Rate: 100.0%
Epoch: 195 D Loss: 0.947257 G Loss: 1.02785  True Positive Rate: 83.3% True Negative Rate: 100.0%
Epoch: 196 D Loss: 0.924251 G Loss: 1.32346  True Positive Rate: 66.7% True Negative Rate: 100.0%
Epoch: 197 D Loss: 0.927951 G Loss: 1.06843  True Positive Rate: 56.7% True Negative Rate: 100.0%
Epoch: 198 D Loss: 0.936711 G Loss: 1.21217  True Positive Rate: 63.3% True Negative Rate: 100.0%
Epoch: 199 D Loss: 0.937629 G Loss: 1.2542   True Positive Rate: 60.0% True Negative Rate: 100.0%
/usr/local/lib/python3.6/dist-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)
end_time :  -34.345648765563965
~~~